In [ ]:
class KalmanFilter(object):
  def __init__(self, A = None, B = None, H = None, Q = None, R = None, P = None, x0 = None):
    if(A is None or H is None):
      raise ValueError("Set proper system dynamics.")

    #Number of state variables
    self.n = A.shape[1]

    #Number of measurement variables
    self.m = H.shape[1]

    self.A = A
    self.H = H
    self.B = np.zeros((1,1)) if B is None else B

    #Covariance is often a diagonal matrix since variables are independent
    self.Q = np.eye(self.n) if Q is None else Q
    self.R = np.eye(self.n) if R is None else R
    self.P = np.eye(self.n) if P is None else P
    self.x = np.zeros((self.n, 1)) if x0 is None else x0

  def predict(self, u = None):
    if u is None: u = np.zeros((self.B.shape[1],1))
                               
    self.x = (self.A @ self.x) + (self.B@ u)
    self.P = ((self.A @self.P)@ self.A.T) + self.Q
    return self.x

  def update(self, z):
    S = self.R + (self.H @ (self.P @ self.H.T)) # in parenthesees of inverse

    #This is a Cholesky Solver which uses the fact that S is symmetric positive definite to solve for K quicker
    #We are solving for K given that S^T * K^T = H * P^T
    K = sp.linalg.cho_solve(sp.linalg.cho_factor(S.T),(self.H @ self.P.T)).T

    #This is the equation for K in the description of the filter for reference
    # K = ((self.P @ self.H.T) @ np.linalg.inv(S))

    y = z - (self.H @ self.x) # in parenthesees of inverse
    self.x = self.x + (K@ y)

    I = np.eye(self.n)
    self.P = (((I-(K@self.H))@ self.P)@((I-(K@self.H)).T))+(K @(self.R @K.T))
    return K